In [1]:
from pipeline import *

path = r"C:\Users\Levi\Desktop\localized-clock-roi-ds-raw-files\raw-videos\broadcast_12.mp4"
extract_timestamps_from_video(path, "test.json")


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
  0%|          | 101/30411 [00:24<2:02:06,  4.14it/s]


# Semi-Supervised DB Expansion
***

In [ ]:
for row in predictions:
    # Convert row values to strings and join them with a space
    row_string = ' '.join([f'{value:.4f}' for value in row])
    print(row_string)